In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.1'
spark_version = 'spark-3.3.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:2 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:4 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:5 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:6 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease [24.3 kB]
Hit:7 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:9 http://archive.ubuntu.com/ubuntu focal InRelease
Get:10 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:11 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2,590 kB]
Get:12 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:13 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1,028 kB]
Get:

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.1.2/22-big-data/home_sales_revised.csv"



In [4]:
# 2. Create a temporary view of the DataFrame.
spark.sparkContext.addFile(url)
temp_df= df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True, quote="\"", escape="\"")

temp_df.show()


+--------------------+-------------------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|               date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+-------------------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08 00:00:00|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13 00:00:00|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12 00:00:00|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16 00:00:00|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08 00:00:00|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa0052

In [5]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
temp_df.createOrReplaceTempView("home_sales")

# Execute a SQL query to calculate the average price for a four bedroom house sold in each year
result = spark.sql("SELECT YEAR(date) AS Year, ROUND(AVG(price), 2) AS Avg_Price FROM home_sales WHERE bedrooms = 3 AND bathrooms = 3 AND sqft_living >= 2000  GROUP BY Year ORDER BY Year")
#Show results
result.show()


+----+---------+
|Year|Avg_Price|
+----+---------+
|2019|294372.02|
|2020| 296314.6|
|2021|305118.97|
|2022|302871.28|
+----+---------+



In [6]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
temp_df.createOrReplaceTempView("home_sales")

# Execute a SQL query to calculate the average price for a 3 bedroom & 3 bath house sold in each year
result = spark.sql("SELECT YEAR(date) AS Year, ROUND(AVG(price), 2) AS Avg_Price FROM home_sales WHERE bedrooms = 3 and bathrooms = 3 GROUP BY Year ORDER BY Year")

# Show the results
result.show()


+----+---------+
|Year|Avg_Price|
+----+---------+
|2019|287287.82|
|2020|294204.16|
|2021|294211.46|
|2022|292725.69|
+----+---------+



In [7]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

temp_df.createOrReplaceTempView("home_sales")

# Execute a SQL query to calculate the average price for a 3 bedroom, 3 bath with 2000 sq ft+ house sold in each year
result = spark.sql("SELECT YEAR(date) AS Year, ROUND(AVG(price), 2) AS Avg_Price FROM home_sales WHERE bedrooms = 3 AND bathrooms = 3 AND sqft_living >= 2000  GROUP BY Year ORDER BY Year")

# Show the results
result.show()


+----+---------+
|Year|Avg_Price|
+----+---------+
|2019|294372.02|
|2020| 296314.6|
|2021|305118.97|
|2022|302871.28|
+----+---------+



In [8]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.
from pyspark.sql.functions import avg, round
start_time = time.time()

# Create a view of the DataFrame that meets the criteria
temp_df.createOrReplaceTempView("home_sales_view")
view = spark.sql("SELECT *, CASE WHEN price >= 350000 THEN 'High' ELSE 'Low' END AS price_rating FROM home_sales_view")

# Calculate the average price rating
result = view.groupBy("price_rating").agg(round(avg("price"), 2).alias("avg_price"))

# Show the results
result.show()

print("--- %s seconds ---" % (time.time() - start_time))

+------------+---------+
|price_rating|avg_price|
+------------+---------+
|        High|473796.26|
|         Low|235148.14|
+------------+---------+

--- 1.5866479873657227 seconds ---


In [9]:
# 7. Cache the the temporary table home_sales.
temp_df.createOrReplaceTempView("home_sales_view")
spark.table("home_sales_view").cache()

DataFrame[id: string, date: timestamp, date_built: int, price: int, bedrooms: int, bathrooms: int, sqft_living: int, sqft_lot: int, floors: int, waterfront: int, view: int]

In [10]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [11]:
# 9. Using the cached data, run the query that filters out the view ratings with average price of greater than $350,000. 
# Determine the runtime and compare it to uncached runtime.

start_time = time.time()
result = spark.sql("SELECT view, ROUND(AVG(price), 2) AS avg_price FROM home_sales WHERE price < 350000 GROUP BY view")

print("--- %s seconds ---" % (time.time() - start_time))


--- 0.04801344871520996 seconds ---


In [12]:
# 10. Partition the home sales dataset by the date_built field.
temp_df.write.parquet('parquet_date_basic', mode='overwrite')

In [13]:
# 11. Read the formatted parquet data.
parquet_df=spark.read.parquet('parquet_date_basic')

In [14]:
# 12. Create a temporary table for the parquet data.
parquet_df.createOrReplaceTempView('p_date')

In [15]:
# 13. Run the query that filters out the view ratings with average price of greater than $350,000 with the parquet DataFrame. 
# Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version. 

start_time = time.time()

result = spark.sql("SELECT view, ROUND(AVG(price), 2) AS avg_price FROM home_sales WHERE price < 350000 GROUP BY view")

print("--- %s seconds ---" % (time.time() - start_time))

--- 0.03904986381530762 seconds ---


In [16]:
# 14. Uncache the home_sales temporary table.
spark.table("home_sales_view").unpersist()

DataFrame[id: string, date: timestamp, date_built: int, price: int, bedrooms: int, bathrooms: int, sqft_living: int, sqft_lot: int, floors: int, waterfront: int, view: int]

In [17]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales_view')


False